In [14]:
import pandas as pd

df_manuf_brecha = pd.read_csv('../results/bea/majority_owned_nonbank/brecha_productividad_all_sectors.csv',
                              sep=" ")
df_manuf_brecha = df_manuf_brecha[['country', 'sector', 'year', 
                                     'ipt_arg_99_index',  'ipt_bench_99_index', 
                                     'brecha_anio_base' ]]
df = df_manuf_brecha.sort_values(by=['country', 'sector', 'year' ])


df.sample(4)

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base
104,Germany,Computers and electronic products,1999,1.0,1,0.232213
59,Greece,Wholesale Trade,1999,1.0,1,0.271249
110,Finland,Transportation Equipment,1999,1.0,1,0.161808
136,Ireland,Finance and insurance,1999,1.0,1,0.210383


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164 entries, 34 to 58
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             164 non-null    object 
 1   sector              164 non-null    object 
 2   year                164 non-null    int64  
 3   ipt_arg_99_index    154 non-null    float64
 4   ipt_bench_99_index  164 non-null    int64  
 5   brecha_anio_base    154 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 9.0+ KB


In [16]:
# Función para calcular la brecha
def calcular_brecha(row):
    if pd.notna(row['brecha_anio_base']):
        return row['brecha_anio_base']
    else:
        filtro_anterior = (df['year'] == row['year'] - 1) & (df['country'] == row['country']) & (df['sector'] == row['sector'])
        if df[filtro_anterior].shape[0] > 0:
            brecha_anio_anterior = df.loc[filtro_anterior, 'brecha_calculada'].values[0]
            ipt_arg_ratio = row['ipt_arg_99_index'] / df.loc[filtro_anterior, 'ipt_arg_99_index'].values[0]
            ipt_bench_ratio = row['ipt_bench_99_index'] / df.loc[filtro_anterior, 'ipt_bench_99_index'].values[0]
            return brecha_anio_anterior * (ipt_arg_ratio / ipt_bench_ratio)
        else:
            return None


In [17]:

# # Iterar sobre las filas y calcular las brechas
for index, row in df.iterrows():
    df.at[index, 'brecha_calculada'] = calcular_brecha(row)

df.head()

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
34,Denmark,All Industries Total,1999,1.0,1,0.435054,0.435054
158,Denmark,Electrical equipment,1999,NaN,1,NaN,NaN
129,Denmark,Finance and insurance,1999,1.0,1,0.028836,0.028836
92,Denmark,Food,1999,1.0,1,0.198362,0.198362
149,Denmark,Manufacturing without Transportation Equipment,1999,1.0,1,0.466805,0.466805


In [18]:
# Agrupar por país y aplicar el cálculo
resultados_por_pais = pd.DataFrame()

for country, group_df in df.groupby(['country', 'sector']):
    group_df['brecha_calculada'] = group_df.apply(calcular_brecha, axis=1)
    resultados_por_pais = pd.concat([resultados_por_pais, group_df])

# Mostrar el DataFrame resultante
resultados_por_pais.head()

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
34,Denmark,All Industries Total,1999,1.0,1,0.435054,0.435054
158,Denmark,Electrical equipment,1999,NaN,1,NaN,NaN
129,Denmark,Finance and insurance,1999,1.0,1,0.028836,0.028836
92,Denmark,Food,1999,1.0,1,0.198362,0.198362
149,Denmark,Manufacturing without Transportation Equipment,1999,1.0,1,0.466805,0.466805


In [19]:
resultados_por_pais.to_csv('../results/bea/majority_owned_nonbank/productivity_gap_all_sectors_sep.csv',
          index   = False,
          sep=';', decimal=',')
resultados_por_pais.to_csv('../results/bea/majority_owned_nonbank/productivity_gap_all_sectors.csv',
          index   = False)

In [20]:
resultados_por_pais[resultados_por_pais['country']=='Germany'].head()

,country,sector,year,ipt_arg_99_index,ipt_bench_99_index,brecha_anio_base,brecha_calculada
33,Germany,All Industries Total,1999,1.0,1,0.412135,0.412135
47,Germany,Chemicals,1999,1.0,1,0.312805,0.312805
104,Germany,Computers and electronic products,1999,1.0,1,0.232213,0.232213
160,Germany,Electrical equipment,1999,NaN,1,NaN,NaN
135,Germany,Finance and insurance,1999,1.0,1,0.205707,0.205707
